In [1]:
from langchain_community.document_loaders import ImageCaptionLoader
from tempfile import NamedTemporaryFile

from langchain.agents import initialize_agent
from langchain.chat_models import ChatOllama
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from langchain.tools import BaseTool
from transformers import BlipProcessor, BlipForConditionalGeneration, DetrImageProcessor, DetrForObjectDetection
from PIL import Image
import torch

from langchain.callbacks import StreamingStdOutCallbackHandler

/home/ga111o/document/MarkDown/kwu-idea-lab/projects/add-alt-using-llm/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
list_image_urls = [
    "0.jpg",
    "2.jpg"
]

In [7]:
# loader = ImageCaptionLoader(path_images=list_image_urls)
loader = ImageCaptionLoader(images=list_image_urls)
list_docs = loader.load()
list_docs

/home/ga111o/document/MarkDown/kwu-idea-lab/projects/add-alt-using-llm/env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ga111o/document/MarkDown/kwu-idea-lab/projects/add-alt-using-llm/env/lib/python3.11/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[Document(page_content="an image of a girl with a sword and text that reads, i'ver, i", metadata={'image_path': '0.jpg'}),
 Document(page_content='an image of a book with a fish on it [SEP]', metadata={'image_path': '2.jpg'})]